<a href="https://colab.research.google.com/github/shettyashraya6-boop/practice/blob/main/Project_cdac_imp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyarrow

In [ ]:
!pip install --upgrade pyarrow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 21.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import dask.dataframe as dd
df = dd.read_csv(
    r"/content/drive/MyDrive/Project_final (1)/Project_cdac.csv",
    engine="python",
    on_bad_lines="skip"
)


In [ ]:
df.head()

,Consumer_complaint,Sub_category
0,My credit report contains incorrect and mislea...,Credit reporting
1,Be sure that disputed accounts are taken down ...,Credit reporting
2,I recently obtained my credit report and found...,Credit reporting
3,I am extremely disappointed and frustrated to ...,I do not know
4,These are not my accounts.,Credit reporting


In [ ]:
df.columns

Index(['Consumer_complaint', 'Sub_category'], dtype='object')

In [ ]:
df.isnull().sum()

Dask Series Structure:
npartitions=1
Consumer_complaint    int64
Sub_category            ...
Dask Name: sum, 5 expressions
Expr=(~ NotNull(frame=ArrowStringConversion(frame=FromMapProjectable(33babaa)))).sum()

In [ ]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
#df = df.reset_index(drop=True)
#df.to_parquet("clean.parquet")

In [ ]:
df = dd.read_parquet('/content/clean.parquet')


ValueError: An error occurred while calling the read_parquet method registered to the pandas backend.
Original Message: No files satisfy the `parquet_file_extension` criteria (files must end with ('.parq', '.parquet', '.pq')).

In [ ]:
df.shape[0].compute()

12302999

In [ ]:
df.columns

Index(['Consumer_complaint', 'Sub_category'], dtype='object')

In [ ]:
df=df.rename(columns={'Consumer_complaint':'text','Sub_category':'label'})

### REMOVE SHORT COMPLAINTS

In [ ]:
#\S+ = one or more non-whitespace characters

df['word_count'] = df['text'].str.count(r'\S+')
df = df[df['word_count'] >= 15].drop(columns=['word_count'])

In [ ]:
df.shape[0].compute()

2574208

## LOWERCASE

In [ ]:
df['text'] = df['text'].str.lower()

## REMOVE URLS, EMAILS, NUMBERS and special data specific characters

re.sub(pattern, replacement, string, count=0, flags=0)
Used for substituting occurrences of a regular expression pattern in a string with a specified replacement

Meta=metadata
column names
dtypes
structure of detail

Dask is lazy, it needs this info in advance to build the task graph.
In Dask:
nothing runs yet
Dask cannot inspect real data
so it asks:
“What will this function return?”
If you don’t tell it, Dask may:
guess incorrectly
throw warnings/errors
build a wrong graph

In [ ]:

def remove_noise(text):
    text = re.sub(r'http\S+|www\S+', '', text)   # remove URLs
    text = re.sub(r'\S+@\S+', '', text)          # remove emails
    text = re.sub(r'\d+', '', text)  # remove numbers
    text=re.sub(r"(#+|X+)",'',text)  #remove cpfb specific hidden signs
    text=re.sub(r"[^\w\s]",'', text) #remove punctuation
    return text
df['text'] = df['text'].apply(remove_noise, meta=('text', 'str'))


In [ ]:
df.shape[0].compute()

2574208

In [ ]:
df.head(10)

,text,label
2,i recently obtained my credit report and found...,Credit reporting
3,i am extremely disappointed and frustrated to ...,I do not know
5,i do not recognize the aforementioned accounts...,Credit reporting
6,i did not approve these transactions to be inc...,Credit reporting
7,kindly address this issue on my credit report ...,Credit reporting
8,i am submitting this dispute statement for my ...,Credit reporting
9,i am formally disputing the accuracy of the fo...,Credit reporting
10,i am filing a complaint because my creditor is...,Credit reporting
11,dear cfpb we are filing a formal complaint aga...,VA mortgage
12,there are xxxx collections being reported to t...,Credit reporting


## REMOVE CFPB BOILERPLATE TEXT

Boilerplate text is a term used to describe standardized, repeated, or template-like text that appears in documents, emails, or websites. It’s usually not specific to the particular situation but is included as a generic, pre-written block.

In [ ]:
boilerplate_patterns = [
    r'i am writing to file a complaint',
    r'this complaint is submitted to the cfpb',
    r'the company failed to respond',
    r'i would like to file a complaint',
]

def remove_boilerplate(text):
    for pattern in boilerplate_patterns:
        text = re.sub(pattern, '', text)
    return text

df['text'] = df['text'].apply(remove_boilerplate,meta=('text','str'))


In [ ]:
df.shape[0].compute()

2574208

In [ ]:
df.head(10)

,text,label
2,i recently obtained my credit report and found...,Credit reporting
3,i am extremely disappointed and frustrated to ...,I do not know
5,i do not recognize the aforementioned accounts...,Credit reporting
6,i did not approve these transactions to be inc...,Credit reporting
7,kindly address this issue on my credit report ...,Credit reporting
8,i am submitting this dispute statement for my ...,Credit reporting
9,i am formally disputing the accuracy of the fo...,Credit reporting
10,i am filing a complaint because my creditor is...,Credit reporting
11,dear cfpb we are filing a formal complaint aga...,VA mortgage
12,there are xxxx collections being reported to t...,Credit reporting


## REMOVE PUNCTUATION

re.sub(r'[^\w\s]', ' ', x) identifies any character that is NOT a letter, number, underscore, or space and replaces it with a single space.


map_partitions with a lambda (more like pandas apply)

In [ ]:
#df["text"] = df["text"].map_partitions(lambda s: s.apply(lambda x: re.sub(r"[^\w\s]",'', x))



In [ ]:
df.shape[0].compute()

3034922

To save(use this for bert)

engine	Which library writes Parquet (pyarrow or fastparquet)
write_index	Whether to save the index column (__null_dask_index__)
compression	Compresses data to save space & speed I/O (snappy, gzip, etc.)

To Read
df2 = dd.read_parquet("output_folder")

In [ ]:
# Define the path in your Google Drive
drive_path = '/content/drive/MyDrive/Project_final/Bert.parquet' # Adjust path as needed

# Save the DataFrame
df.to_parquet(drive_path)
print(f"File saved to: {drive_path}")


File saved to: /content/drive/MyDrive/Project_final/Bert.parquet


In [ ]:
df.head(10)

,text,label
2,i recently obtained my credit report and found...,Credit reporting
3,i am extremely disappointed and frustrated to ...,I do not know
5,i do not recognize the aforementioned accounts...,Credit reporting
6,i did not approve these transactions to be inc...,Credit reporting
7,kindly address this issue on my credit report ...,Credit reporting
8,i am submitting this dispute statement for my ...,Credit reporting
9,i am formally disputing the accuracy of the fo...,Credit reporting
10,i am filing a complaint because my creditor is...,Credit reporting
11,dear cfpb we are filing a formal complaint aga...,VA mortgage
12,there are xxxx collections being reported to t...,Credit reporting
